In [87]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import heapq as heap

In [88]:
flights = pd.read_csv('Dataset.csv')

In [89]:
df = flights.drop_duplicates(subset=['SourceAirport', 'DestinationAirport', 'SourceAirport_City', 'SourceAirport_Country',
                               'SourceAirport_Latitude', 'SourceAirport_Longitude', 'SourceAirport_Altitude',
                               'DestinationAirport_City', 'DestinationAirport_Country', 'DestinationAirport_Latitude',
                               'DestinationAirport_Longitude', 'DestinationAirport_Altitude', 'Distance', 'FlyTime', 'Price'])


In [90]:
df = df.reset_index(drop = True)

In [91]:
df

,Airline,SourceAirport,DestinationAirport,SourceAirport_City,SourceAirport_Country,SourceAirport_Latitude,SourceAirport_Longitude,SourceAirport_Altitude,DestinationAirport_City,DestinationAirport_Country,DestinationAirport_Latitude,DestinationAirport_Longitude,DestinationAirport_Altitude,Distance,FlyTime,Price
0,Aerocondor,Sochi International Airport,Kazan International Airport,Sochi,Russia,43.449902,39.956600,89,Kazan,Russia,55.606201,49.278702,411,1506.825604,2.371833,723.412802
1,Aerocondor,Astrakhan Airport,Kazan International Airport,Astrakhan,Russia,46.283298,48.006302,-65,Kazan,Russia,55.606201,49.278702,411,1040.438320,1.792470,490.219160
2,Aerocondor,Chelyabinsk Balandino Airport,Kazan International Airport,Chelyabinsk,Russia,55.305801,61.503300,769,Kazan,Russia,55.606201,49.278702,411,770.508500,1.457153,355.254250
3,Aerocondor,Domodedovo International Airport,Kazan International Airport,Moscow,Russia,55.408798,37.906300,588,Kazan,Russia,55.606201,49.278702,411,715.649350,1.389005,327.824675
4,Aerocondor,Belgorod International Airport,Kazan International Airport,Belgorod,Russia,50.643799,36.590099,735,Kazan,Russia,55.606201,49.278702,411,1008.253110,1.752488,474.126555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36521,Iberworld,Lifou Airport,Tiga Airport,Lifou,New Caledonia,-20.774799,167.240005,92,Tiga,New Caledonia,-21.096100,167.804001,128,68.609280,0.585229,4.304640
36522,Iberworld,Nouméa Magenta Airport,Touho Airport,Noumea,New Caledonia,-22.258301,166.473007,10,Touho,New Caledonia,-20.790001,165.259003,10,205.972279,0.755866,72.986139
36523,Iberworld,Nouméa Magenta Airport,Ouvéa Airport,Noumea,New Caledonia,-22.258301,166.473007,10,Ouvea,New Caledonia,-20.640600,166.572998,23,180.177492,0.723823,60.088746
36524,Iberworld,Lifou Airport,Ouvéa Airport,Lifou,New Caledonia,-20.774799,167.240005,92,Ouvea,New Caledonia,-20.640600,166.572998,23,70.962955,0.588153,5.481478


### Airline                             
### اسم خط هوایی 

### SourceAirport 
### فرودگاه مبدا 

### DestinationAirport  
### فرودگاه مقصد

### SourceAirport_City
### شهر فرودگاه مبدا

### SourceAirport_Country 
### کشور فرودگاه مبدا

### SourceAirport_Latitude          
### عرض جغرافیایی فرودگاه مبدا

### SourceAirport_Longitude                  
### طول جغرافیایی فرودگاه مبدا

### SourceAirport_Altitude            
### ارتفاع جغرافیایی فرودگاه مبدا

### DestinationAirport_City    
### شهر فرودگاه مقصد

### DestinationAirport_Country 
### کشور فرودگاه مقصد

### DestinationAirport_Latitude          
### عرض جغرافیایی فرودگاه مقصد

### DestinationAirport_Longitude            
### طول جغرافیایی فرودگاه مقصد

### DestinationAirport_Altitude       
### ارتفاع جغرافیایی فرودگاه مقصد

### Distance               
### فاصله

### FlyTime               
### زمان پرواز

### Price          
### هزینه

In [92]:
df['SourceAirport_Location'] = df.apply(lambda row: (row['SourceAirport_Latitude'], row['SourceAirport_Longitude'], row['SourceAirport_Altitude']), axis=1)

# Drop the individual columns since you now have a single combined column
df.drop(['SourceAirport_Latitude', 'SourceAirport_Longitude', 'SourceAirport_Altitude'], axis=1, inplace=True)


In [93]:
# Create a new column 'DestinationAirport_Location' by combining the latitude, longitude, and altitude into a tuple
df['DestinationAirport_Location'] = df.apply(lambda row: (row['DestinationAirport_Latitude'], row['DestinationAirport_Longitude'], row['DestinationAirport_Altitude']), axis=1)

# Drop the individual columns since you now have a single combined column
df.drop(['DestinationAirport_Latitude', 'DestinationAirport_Longitude', 'DestinationAirport_Altitude'], axis=1, inplace=True)


In [94]:
def haversine(lat1, lon1, alt1, lat2, lon2, alt2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Earth radius in meters (you can use different values based on your dataset)
    R = 6371000

    # Altitude difference
    alt_diff = abs(alt1 - alt2)

    # Haversine formula
    a = math.sin((lat2 - lat1) / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin((lon2 - lon1) / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Calculate the distance
    distance = R * c + alt_diff

    return distance

In [95]:
class Node:
    def __init__(self, airport, city, country, coordination):
        self.airport = airport
        self.city = city
        self.country = country
        self.coordination = coordination


In [96]:
class Edge:
    def __init__(self, airline, distance, fly_time, price):
        self.airline = airline
        self.distance = distance
        self.fly_time = fly_time
        self.price = price


In [97]:
df

,Airline,SourceAirport,DestinationAirport,SourceAirport_City,SourceAirport_Country,DestinationAirport_City,DestinationAirport_Country,Distance,FlyTime,Price,SourceAirport_Location,DestinationAirport_Location
0,Aerocondor,Sochi International Airport,Kazan International Airport,Sochi,Russia,Kazan,Russia,1506.825604,2.371833,723.412802,"(43.449902, 39.9566, 89)","(55.606201171875, 49.278701782227, 411)"
1,Aerocondor,Astrakhan Airport,Kazan International Airport,Astrakhan,Russia,Kazan,Russia,1040.438320,1.792470,490.219160,"(46.2832984924, 48.0063018799, -65)","(55.606201171875, 49.278701782227, 411)"
2,Aerocondor,Chelyabinsk Balandino Airport,Kazan International Airport,Chelyabinsk,Russia,Kazan,Russia,770.508500,1.457153,355.254250,"(55.305801, 61.5033, 769)","(55.606201171875, 49.278701782227, 411)"
3,Aerocondor,Domodedovo International Airport,Kazan International Airport,Moscow,Russia,Kazan,Russia,715.649350,1.389005,327.824675,"(55.40879821777344, 37.90629959106445, 588)","(55.606201171875, 49.278701782227, 411)"
4,Aerocondor,Belgorod International Airport,Kazan International Airport,Belgorod,Russia,Kazan,Russia,1008.253110,1.752488,474.126555,"(50.643798828125, 36.5900993347168, 735)","(55.606201171875, 49.278701782227, 411)"
...,...,...,...,...,...,...,...,...,...,...,...,...
36521,Iberworld,Lifou Airport,Tiga Airport,Lifou,New Caledonia,Tiga,New Caledonia,68.609280,0.585229,4.304640,"(-20.774799346923828, 167.24000549316406, 92)","(-21.096099853515625, 167.8040008544922, 128)"
36522,Iberworld,Nouméa Magenta Airport,Touho Airport,Noumea,New Caledonia,Touho,New Caledonia,205.972279,0.755866,72.986139,"(-22.25830078125, 166.47300720214844, 10)","(-20.790000915527344, 165.25900268554688, 10)"
36523,Iberworld,Nouméa Magenta Airport,Ouvéa Airport,Noumea,New Caledonia,Ouvea,New Caledonia,180.177492,0.723823,60.088746,"(-22.25830078125, 166.47300720214844, 10)","(-20.640600204467773, 166.572998046875, 23)"
36524,Iberworld,Lifou Airport,Ouvéa Airport,Lifou,New Caledonia,Ouvea,New Caledonia,70.962955,0.588153,5.481478,"(-20.774799346923828, 167.24000549316406, 92)","(-20.640600204467773, 166.572998046875, 23)"


In [98]:
dft = df[['Distance', 'FlyTime', 'Price']]

In [99]:
dft

,Distance,FlyTime,Price
0,1506.825604,2.371833,723.412802
1,1040.438320,1.792470,490.219160
2,770.508500,1.457153,355.254250
3,715.649350,1.389005,327.824675
4,1008.253110,1.752488,474.126555
...,...,...,...
36521,68.609280,0.585229,4.304640
36522,205.972279,0.755866,72.986139
36523,180.177492,0.723823,60.088746
36524,70.962955,0.588153,5.481478


In [100]:
dft.corr()

,Distance,FlyTime,Price
Distance,1.0,1.0,1.0
FlyTime,1.0,1.0,1.0
Price,1.0,1.0,1.0


In [101]:
nodes = {}
graph = {}

In [102]:
for data in df.values:
    if data[1] not in nodes:
        nodes[data[1]] = Node(data[1], data[3], data[4], data[10])
    
    if data[2] not in nodes:
        nodes[data[2]] = Node(data[2], data[5], data[6], data[11])
    
    e = Edge(data[0], data[7], data[8], data[9])
    
    if data[1] not in graph:
        graph[data[1]] = {}
    
    if data[2] not in graph:
        graph[data[2]] = {}
        
    graph[data[1]][data[2]] = e

In [103]:
nodes

{'Sochi International Airport': <__main__.Node at 0x7f3f152b1de0>,
 'Kazan International Airport': <__main__.Node at 0x7f3f152b0700>,
 'Astrakhan Airport': <__main__.Node at 0x7f3f0fe8c490>,
 'Chelyabinsk Balandino Airport': <__main__.Node at 0x7f3f0fe8ceb0>,
 'Domodedovo International Airport': <__main__.Node at 0x7f3f1b7a8850>,
 'Belgorod International Airport': <__main__.Node at 0x7f3f1b7aa110>,
 'Pulkovo Airport': <__main__.Node at 0x7f3f1b7a9060>,
 'Koltsovo Airport': <__main__.Node at 0x7f3f1b7a8ee0>,
 'Fergana International Airport': <__main__.Node at 0x7f3f1b7a84f0>,
 'Samarkand Airport': <__main__.Node at 0x7f3f1b7a8be0>,
 'Tashkent International Airport': <__main__.Node at 0x7f3f1b7a9840>,
 'Sheremetyevo International Airport': <__main__.Node at 0x7f3f1b7a9930>,
 'Dushanbe Airport': <__main__.Node at 0x7f3f1b7a8640>,
 'Khudzhand Airport': <__main__.Node at 0x7f3f1b7aa5f0>,
 'Atatürk International Airport': <__main__.Node at 0x7f3f1b7aa290>,
 'Sharjah International Airport': <

In [104]:
len(nodes)

3113

In [105]:
graph

{'Sochi International Airport': {'Kazan International Airport': <__main__.Edge at 0x7f3f0fe8dc90>,
  'Domodedovo International Airport': <__main__.Edge at 0x7f3f0fea64a0>,
  'Pulkovo Airport': <__main__.Edge at 0x7f3f0febc5b0>,
  'Koltsovo Airport': <__main__.Edge at 0x7f3f0febfbe0>,
  'Dushanbe Airport': <__main__.Edge at 0x7f3f0fec7400>,
  'Khudzhand Airport': <__main__.Edge at 0x7f3f1b78f880>,
  'Atatürk International Airport': <__main__.Edge at 0x7f3f0fec93c0>,
  'Chişinău International Airport': <__main__.Edge at 0x7f3f0f841900>,
  'Minsk National Airport': <__main__.Edge at 0x7f3f0f88d060>,
  'Zvartnots International Airport': <__main__.Edge at 0x7f3f0f8bc880>,
  'Krasnodar Pashkovsky International Airport': <__main__.Edge at 0x7f3f0f8c7460>,
  'Tashkent International Airport': <__main__.Edge at 0x7f3f0f8edf00>,
  'Yemelyanovo Airport': <__main__.Edge at 0x7f3f1e40c5e0>,
  'Trabzon International Airport': <__main__.Edge at 0x7f3f1e3f7580>,
  'Omsk Central Airport': <__main__.Edge

In [106]:
len(graph)

3113

In [ ]:
def dijkstra_score(e : Edge):
    return e.distance + e.fly_time * 700 + price * 2

In [ ]:
def dijkstra(graph):
    